# Indexing and slicing a rolling (circular) series

Inspired by this stackoverflow question:
 - https://stackoverflow.com/questions/72876190/rolling-windows-in-pandas-how-to-wrap-around-with-datetimeindex/72876308#72876291

In [1]:
import numpy as np
import pandas as pd

data = pd.Series(index=pd.date_range('2022-1-1', '2022-12-31', freq='D'),
                 data=np.random.random(365))

In [2]:
n = 3
rolling_sums1 = data.rolling(n).sum()
rolling_sums1

2022-01-01         NaN
2022-01-02         NaN
2022-01-03    0.124780
2022-01-04    0.615610
2022-01-05    1.420577
                ...   
2022-12-27    0.572875
2022-12-28    0.839732
2022-12-29    0.997892
2022-12-30    1.681717
2022-12-31    1.787752
Freq: D, Length: 365, dtype: float64

In [3]:
rolling_sums = (pd.concat([data[-n:], data])).rolling(n).sum()[n:]
rolling_sums

2022-01-01    1.521914
2022-01-02    0.751475
2022-01-03    0.124780
2022-01-04    0.615610
2022-01-05    1.420577
                ...   
2022-12-27    0.572875
2022-12-28    0.839732
2022-12-29    0.997892
2022-12-30    1.681717
2022-12-31    1.787752
Length: 365, dtype: float64

In [4]:
# Test
assert(rolling_sums.loc["2022-01-01"] ==
    data[["2022-12-30", "2022-12-31", "2022-01-01"]].sum())

## Rolling slice functions

In case you also want calculate individual rolling sums for certain days there are some Python slicing tricks that can be useful.

First, consider this rolling list slicer:

In [5]:
def rolling_slice(items, i, n):
    i = i % len(items)
    return (items[i:] + items[:i])[:n]

assert rolling_slice([1, 2, 3], 0, 3) == [1, 2, 3]
assert rolling_slice([1, 2, 3], 1, 3) == [2, 3, 1]
assert rolling_slice([1, 2, 3], -1, 3) == [3, 1, 2]

Now, adapt this for Pandas series and dataframes:

However, these functions are not an efficient way to calculate all the rolling sums.

In [6]:
def rolling_slice_pd(data, i, n):
    i = i % len(data)
    return pd.concat([data.iloc[i:], data.iloc[:i]]).iloc[:n]

i = 0  # 2022-01-01
print(rolling_slice_pd(data, i-n+1, n))

2022-12-30    0.830060
2022-12-31    0.647399
2022-01-01    0.044455
dtype: float64


In [7]:
print(rolling_slice_pd(data, i-n+1, n).sum())

1.5219143677750582
